# Create datasets
### Test set is defined af part of the original training datset; Due to hidden labels in the original test set provided by HuggingFace

In [6]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
from more_itertools import roundrobin
import pyarrow as pa
import pandas as pd
from datasets import load_from_disk

In [3]:
# Import dataset from HuggingFace
dataset = load_dataset("sst2")

Dataset sst2 downloaded and prepared to /zhome/94/5/127021/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 475.98it/s]


In [4]:
# validation set
validation_dataset = dataset['validation']

In [5]:
# Check distribution in validation set
validation_dataset['label'].count(0) / validation_dataset['label'].count(1)

# Original test set size: 1821
# Thus, amount of 0-labels should be 895, while 1-labels should be 926

0.963963963963964

In [8]:
dataset["train"]["label"][0]

0

In [10]:
# load
datadir = '/work3/s174498/sst2_dataset/'
test_dataset = load_from_disk(datadir + 'test_dataset')

In [13]:
test_dataset["label"][0]

0

In [7]:
# Sample test set from original train data
# Using the same distribution as seen in the validation set

dataset_test_idx_0 = []
dataset_test_sentence_0 = []
dataset_test_label_0 = []

dataset_test_idx_1 = []
dataset_test_sentence_1 = []
dataset_test_label_1 = []

N = len(test_dataset["label"])

for i in tqdm(list(range(0,N))):
    if (test_dataset["label"][i] == 0):
        if len(dataset_test_label_0) < 895:
            dataset_test_idx_0.append(dataset["train"]["idx"][i])
            dataset_test_sentence_0.append(dataset["train"]["sentence"][i])
            dataset_test_label_0.append(dataset["train"]["label"][i])
    if (dataset["label"][i] == 1):
        if len(dataset_test_label_1) < 926:
            dataset_test_idx_1.append(dataset["train"]["idx"][i])
            dataset_test_sentence_1.append(dataset["train"]["sentence"][i])
            dataset_test_label_1.append(dataset["train"]["label"][i])
    

 99%|█████████▉| 1989/2000 [06:42<00:02,  4.94it/s]


In [12]:
# Gather lists containing labels 0 and 1
dataset_test_idx = list(roundrobin(dataset_test_idx_0,dataset_test_idx_1))
dataset_test_sentence = list(roundrobin(dataset_test_sentence_0,dataset_test_sentence_1))
dataset_test_label = list(roundrobin(dataset_test_label_0,dataset_test_label_1))

In [9]:
# lists as dictionary
test_df = pd.DataFrame({'idx': dataset_test_idx, 'sentence': dataset_test_sentence, 'label': dataset_test_label})

# convert to Huggingface dataset
test_dataset = Dataset(pa.Table.from_pandas(test_df))

In [13]:
# The rest of the dataset is defined as train
train_dataset = dataset["train"].shuffle(seed=42).select([i for i in list(range(len(test_dataset)+1,len(dataset["train"])))]) # 
test_dataset = test_dataset.shuffle(seed=42).select([i for i in list(range(0, len(test_dataset)))])

Loading cached shuffled indices for dataset at /zhome/a6/6/127219/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-da59dc4ea8815b88.arrow


In [14]:
# Check distributions
print(test_dataset['label'].count(0) / test_dataset['label'].count(1))
print(validation_dataset['label'].count(0) / validation_dataset['label'].count(1))
print(train_dataset['label'].count(0) / train_dataset['label'].count(1))

0.9665226781857451
0.963963963963964
0.7926572374360518


In [15]:
# save datasets
train_dataset.save_to_disk('/work3/s174498/sst2_dataset/train_dataset')
validation_dataset.save_to_disk('/work3/s174498/sst2_dataset/validation_dataset')
test_dataset.save_to_disk('/work3/s174498/sst2_dataset/test_dataset')

Loading cached processed dataset at /zhome/a6/6/127219/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-137fd86d64e982b4.arrow


Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]